In [ ]:
%pip install ipython-autotime

In [ ]:
import pandas as pd
import warnings

%load_ext autotime
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/new-york-housing-market/NY-House-Dataset.csv', encoding='utf-8')

# EDA

In [ ]:
df

In [ ]:
df.info()

I'm going to convert the feature names to lowecase because I like them this way. And convert BATH and PROPERTYSQFT to int.

In [ ]:
columns_original = list(df.columns)
columns_lower = list(map(lambda x: x.lower(), columns_original))
columns_new = {key: value for (key, value) in zip(columns_original, columns_lower)}
df.rename(columns=columns_new, inplace=True)

In [ ]:
df = df.astype({"bath": int, "propertysqft": int})

In [ ]:
df[['price', 'beds', 'bath', 'propertysqft', 'latitude', 'longitude']].describe()

## Location features

In [ ]:
import re

It seems to me that this dataset contains redundant features. I'm referring to those about the location of the houses. I'm going to take a closer look on these features.

In [ ]:
df[['address', 'state', 'main_address', 'administrative_area_level_2', 'locality', 'sublocality', 'street_name', 'long_name', 'formatted_address']].sample(5)

In fact there are redundant features here.
* **administrative_area_level_2 and 'locality:** I'm going to keep 'locality' because it looks cleaner and the county is important to the analysis.
* **address, formatted_address and main_address:** I'm going to keep 'formatted_address' because as the name says the addresses are formated and will be easier to get some informations with split(',') function.
* **state and street_name:** I'm going to keep state because the zip code may be important.


In [ ]:
df[['state', 'locality', 'sublocality', 'long_name', 'formatted_address']].sample(10)

The numbers in 'state' are the zip codes and consulting the [United States Zip Codes](https://www.unitedstateszipcodes.org/) website I've found out that the names before the comma was meant to be the post office because there are a lot of matches between the numbers and post office names. But still there are a lot of inconsistences not just in this feaure but 'locality', 'sublocality', and 'long_name'. None of them are reliable.

I turns out that I'm going to extract the street names and zip codes from 'formatted_address' and remove them all.

### Features creation

Usually I remove and add attributes in the feature engineering step but the features I want to add will help the data visualization and analysis. Since I'm adding, I'm going to remove the unwanted ones now too.

In [ ]:
df['zip_code'] = df['formatted_address'].apply(lambda address: int(re.findall(r"\d{5}", address)[0]))

I really don't know if this is cheating but I've found a file on github that seems much more realiable regarding zip code-borough than the features from this dataset.

In [ ]:
url = 'https://github.com/erikgregorywebb/nyc-housing/blob/master/Data/nyc-zip-codes.csv?raw=true'
zip_codes = pd.read_csv(url)
zip_codes = zip_codes.rename(columns={'Borough':'borough', 'Neighborhood':'neighborhood', 'ZipCode':'zip_code'})
zip_codes.sample(5)

In [ ]:
df = df.merge(zip_codes, on='zip_code')
df[['formatted_address', 'zip_code', 'borough', 'neighborhood']].sample(5)

In [ ]:

df['street_name'] = df['formatted_address'].apply(lambda address: address.split(',')[0])
df[['formatted_address', 'zip_code', 'borough', 'neighborhood', 'street_name']].sample(5)

In [ ]:
df = df.drop(['address', 'state', 'main_address', 'administrative_area_level_2', 'locality', 'sublocality', 'long_name', 'formatted_address'], axis=1)
df.head(5)

For anyone reading this notebook I would like to suggest the [geopy](https://geopy.readthedocs.io/en/stable/#nominatim) module. With it you can obtain more geographic information by informing latitude and longitude.

## Data Visualization

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

import matplotlib.pyplot as plt
import seaborn           as sns

colors = sns.color_palette()
%matplotlib inline

To investigate geographical patterns to identify areas with higher or lower property prices I'm going to group the continuous values of 'price' into contiguous intervals. This process will discretize the numbers.

In [ ]:
kbd = KBinsDiscretizer(n_bins=10, encode='ordinal')
price_disc = kbd.fit_transform(df['price'].to_numpy().reshape(-1, 1)).ravel()

sns.set(rc={'figure.figsize':(15, 5)})
sns.scatterplot(data=df, x='longitude', y='latitude', hue=price_disc, palette=sns.color_palette("crest", as_cmap=True), sizes=(10, 200))
plt.tight_layout()
plt.show()

**Conclusion:** I wasn't able to see any useful patterns here.

### Borough x Average Price

In [ ]:
col = 'borough'
average_prices = df.groupby(col)['price'].mean().sort_values(ascending=False).index

sns.set(rc={'figure.figsize':(12, 4)})
sns.set(style="whitegrid")
axs = sns.barplot(data=df, x=col, y='price', order=average_prices, palette='viridis', errorbar=None)
axs.set(yticklabels=[])
plt.bar_label(axs.containers[0])
plt.tight_layout()
plt.show()

### Neighborhood x Average Price

In [ ]:
col = 'neighborhood'
average_prices = df.groupby(col)['price'].mean().sort_values(ascending=False).index

sns.set(rc={'figure.figsize':(12, 6)})
axs = sns.barplot(data=df, x=col, y='price', order=average_prices, palette='viridis', errorbar=None)
axs.set_xticklabels(axs.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

### Square Footage x Price

In [ ]:
sns.set(rc={'figure.figsize':(12, 5)})
axs = sns.lineplot(data=df, x='propertysqft', y='price')
plt.tight_layout()
plt.show()

In [ ]:
# zooming in the plot keeping only properties smaller than 10_000
mask = df['propertysqft'] < 10_000
df_ = df[mask]

sns.set(rc={'figure.figsize':(12, 7)})
axs = sns.lineplot(data=df_, x='propertysqft', y='price')
plt.tight_layout()
plt.show()

There is too much noise! I'm going to discretize 'propertysqft' attribute too. Maybe with price_disc I can find a pattern.

In [ ]:
kbd = KBinsDiscretizer(n_bins=5, encode='ordinal')
sqft_disc = kbd.fit_transform(df['propertysqft'].to_numpy().reshape(-1, 1)).ravel()

sns.set(rc={'figure.figsize':(12, 5)})
axs = sns.lineplot(x=sqft_disc, y=price_disc)
plt.tight_layout()
plt.show()

**Conclusion:** Despite all the noise the data appears to have an ascending trend as property sizes increase.

### Property Type x Average Price

In [ ]:
col = 'type'
average_prices = df.groupby(col)['price'].mean().sort_values(ascending=False).index

sns.set(rc={'figure.figsize':(12, 6)})
sns.set(style="whitegrid")
axs = sns.barplot(data=df, x=col, y='price', order=average_prices, palette='viridis', errorbar=None)
axs.set(yticklabels=[])
axs.set_xticklabels(axs.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

**Conlusion:** Looks like there are some noise here. The values that make sense to me for 'type' are:
* ['Townhouse', 'House', 'Condo', 'Land', 'Multi-family home', 'Mobile house', 'Co-op']

According with [Bankrate](https://www.bankrate.com/real-estate/contingent-meaning/) 'Contingent' means:
> the seller has accepted an offer, but is opting to keep the listing active while they make sure all conditions are properly met

According with [CapitalOne](https://www.capitalone.com/learn-grow/life-events/what-does-pending-mean-in-real-estate/) 'Pending' means:
> a property offer has been accepted and the contingencies met, but the sale hasn't been finalized yet

According with [Rocket Mortgage](https://www.rocketmortgage.com/learn/foreclosure-definition) 'Foreclosure' means:
> is a process that begins when a borrower fails to make their mortgage payments

Ok, here what I'm going to do on the feature engineering step:
* Assuming all properties on this dataset are for sale I'm going to create new binary features with 'foreclosure', 'contigent' ,'coming soon' and 'pending'.
* Rename 'condop' to 'condo' because probably is misstyping.
* Rename 'Foreclosure', 'Contigent' ,'Coming Soon', 'Pending' and 'For sale' to 'Unknown'

### Beds x Average Price

In [ ]:
col = 'beds'

mask = df[col] < 12 # by calling value_counts() I could see there are very few instances with more than 12 beds so I'm ignoring them now
df_ = df[mask]

average_prices = df_.groupby(col)['price'].mean().sort_values(ascending=False).index

sns.set(rc={'figure.figsize':(12, 4)})
sns.set(style="whitegrid")
axs = sns.barplot(data=df_, x=col, y='price', order=average_prices, palette='viridis', errorbar=None)
axs.set(yticklabels=[])
axs.set_xticklabels(axs.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

### Bath x Average Price

In [ ]:
col = 'bath'

mask = df[col] < 10 # by calling value_counts() I could see there are very few instances with more than 10 baths so I'm ignoring them now
df_ = df[mask]

average_prices = df_.groupby(col)['price'].mean().sort_values(ascending=False).index

sns.set(rc={'figure.figsize':(12, 4)})
sns.set(style="whitegrid")
axs = sns.barplot(data=df_, x=col, y='price', order=average_prices, palette='viridis', errorbar=None)
axs.set(yticklabels=[])
axs.set_xticklabels(axs.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

## Statistical test
To analyze the impact of the number of bedrooms and bathrooms on house prices and the broker's influence on the pricing of houses as well I'm going to do a statistical test called Mutual information.

In [ ]:
%pip install category-encoders

In [ ]:
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.feature_selection       import mutual_info_regression

import numpy as np

Bellow you can see the ranking of influence on the house pricing.

In [ ]:
label = df['price'].values
bt_encoded = LeaveOneOutEncoder().fit_transform(df['brokertitle'], label)
matrix     = np.concatenate((bt_encoded, df[['beds', 'bath']].to_numpy()), axis=1)

scores = mutual_info_regression(X=matrix, y=label, discrete_features=[1, 2])
pd.Series(data=scores, index=['brokertitle', 'beds', 'bath']).sort_values(ascending=False)

**Conclusion** The most influential feature in property pricing is 'brokertitle'.

# Train/Test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_set, test_set = train_test_split(df, test_size=0.25)

y_train = train_set['price']
y_test  = test_set['price']

X_train = train_set.drop('price', axis=1).reset_index(drop=True)
X_test  = test_set.drop('price', axis=1).reset_index(drop=True)

y_train.index = X_train.index
y_test.index  = X_test.index

# Feature engineering

In [ ]:
from sklearn.base              import BaseEstimator, TransformerMixin
from sklearn.preprocessing     import OneHotEncoder, StandardScaler

Getting the categorical e numerical feauture names.

In [ ]:
numerical   = list()
categorical = list()

[numerical.append(feature) if df[feature].dtype != 'object' else categorical.append(feature) for feature in df.columns]
numerical.remove('price')

print(categorical)
print(numerical)

## Categorical features

**Step ## -** NaN values - Imputation

In [ ]:
df[categorical].isnull().sum()

**Step ## -** Formating some features.

In [ ]:
X_train['brokertitle'] = X_train['brokertitle'].apply(lambda title: title.replace('Brokered by ', ''))
X_train['type'] = X_train['type'].apply(lambda tpe: tpe.replace(' for sale', ''))

X_test['brokertitle'] = X_test['brokertitle'].apply(lambda title: title.replace('Brokered by ', ''))
X_test['type'] = X_test['type'].apply(lambda tpe: tpe.replace(' for sale', ''))

**Step ## -** Creating 'foreclosure', 'contigent' ,'coming_soon' and 'pending' features from 'type'.

In [ ]:
oe = OneHotEncoder(sparse_output=False)

df_encoded = oe.fit_transform(X_train['type'].to_numpy().reshape(-1, 1))
df_ohot = pd.DataFrame(data=df_encoded, columns=oe.get_feature_names_out())

df_ohot.head(3)

In [ ]:
columns_new = {'x0_Pending':'pending', 'x0_Coming Soon':'coming_soon', 'x0_Contingent':'contingent', 'x0_Foreclosure':'foreclosure'}
df_ohot.rename(columns=columns_new, inplace=True)

cols = ['pending', 'coming_soon', 'contingent', 'foreclosure']
X_train = pd.concat([X_train, df_ohot[cols]], axis=1)

# depending on train_test_split() sometimes you get 'ValueError: Found unknown categories' on the test set
df_encoded = oe.transform(X_test['type'].to_numpy().reshape(-1, 1))

df_ohot = pd.DataFrame(data=df_encoded, columns=oe.get_feature_names_out())
df_ohot.rename(columns=columns_new, inplace=True)
X_test = pd.concat([X_test, df_ohot[cols]], axis=1)

X_train.head(3)

**Step ## -** Renaming those impure values from type to 'Unknown'.

In [ ]:
mask = X_train['type'] == 'Condop'
X_train.loc[mask, 'type'] = 'Condo'

mask = X_test['type'] == 'Condop'
X_test.loc[mask, 'type'] = 'Condo'

In [ ]:
ok_types = ['Townhouse', 'House', 'Condo', 'Land', 'Multi-family home', 'Mobile house', 'Co-op']

mask = X_train['type'].isin(ok_types)
X_train.loc[~mask, 'type'] = 'Unknown'

mask = X_test['type'].isin(ok_types)
X_test.loc[~mask, 'type'] = 'Unknown'

X_train['type'].value_counts()

**Step ## -** Nominal features - Encoding

In [ ]:
loe = LeaveOneOutEncoder(cols=categorical)
X_train = loe.fit_transform(X_train, y_train)
X_test  = loe.transform(X_test)

## Numerical features

**Step ## -** NaN - Imputation

In [ ]:
df[numerical].isnull().sum()

**Step ## -** Binning 'propertysqft' because of the noise.

In [ ]:
kbd = KBinsDiscretizer(n_bins=5, encode='ordinal')
X_train['propertysqft'] = kbd.fit_transform(X_train['propertysqft'].to_numpy().reshape(-1, 1)).ravel()
X_test['propertysqft']  = kbd.transform(X_test['propertysqft'].to_numpy().reshape(-1, 1)).ravel()

**Step ## -** Outliers - Replacement by median.

In [ ]:
class OutliersZScoreReplacer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.mean_std_median = list()
        for name in X.columns:
            mean   = X[name].mean()
            std    = X[name].std()
            median = X[name].median()
            self.mean_std_median.append((mean, std, median))
        return self

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

    def get_feature_names_out(self):
        pass

    def transform(self, X, y=None):
        std_unit = 3
        for index, name in enumerate(X.columns):
            mean    = self.mean_std_median[index][0]
            std     = self.mean_std_median[index][1]
            median  = self.mean_std_median[index][2]
            scores  = ((X[name] - mean) / std)
            filter_mask = ((scores < -std_unit) | (scores > std_unit))
            X.loc[filter_mask, name] = median
        return X

In [ ]:
out = OutliersZScoreReplacer()

df_encoded = out.fit_transform(X_train[numerical])
X_train[df_encoded.columns] = df_encoded

df_encoded = out.transform(X_test[numerical])
X_test[df_encoded.columns] = df_encoded

## All features

**Step ## -** Standardization           

In [ ]:
columns = X_train.columns
std = StandardScaler()

df_encoded = std.fit_transform(X_train[columns])
X_train    = pd.DataFrame(data=df_encoded, columns=std.get_feature_names_out())

df_encoded = std.transform(X_test[columns])
X_test     = pd.DataFrame(data=df_encoded, columns=std.get_feature_names_out())

X_train.head(3)

# Model training

I'm going to test four candidates by using cross validation. The two most promising are gonna go to hyperparameter tuning. I'm going to train the one with the highest score on the full training set. If is necessary because of overfitting I'll decrease it's degree of freedom. The tiebreaker criterion will be the complexity of the model. The simplest wins.

Candidate models:
* Baseline
 * DummyRegressor
* Linear
 * ElasticNet
 * SVR
* Tree based
 * ExtraTreesRegressor
 * XGBRegressor

I'm going to use cross validatoin with neg_mean_squared_error in order to evaluate the candidates except the baseline.

The 'score' parameter of cross_val_score() expects a function that follows the "higher is better" philosophy of Scikit Learn scoring functions. However, when the task is regression, we want the distance between the point calculated (ŷ) and the observed point (y) to be as small as possible. In other words, in this task we would assign 'score' a function that would do the opposite of what it expects. The solution was to multiply the result of the distance between the two points by -1. Therefore the argument is "neg_mean_squared_error".

Since cross_val_score() will return a negative number all I need to do is choose the largest one. Another option is to multiply the result by -1 and choose the model that presents the lowest value.

In [ ]:
from sklearn.base            import clone
from sklearn.dummy           import DummyRegressor
from sklearn.ensemble        import ExtraTreesRegressor
from sklearn.linear_model    import ElasticNet
from sklearn.metrics         import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.svm             import SVR
from xgboost                 import XGBRegressor

In [ ]:
def line_plot(y_true, y_pred, start=0, end=100):
    y_true = y_true[start:end]
    y_pred = y_pred[start:end]
    sns.set(rc={'figure.figsize':(12, 5)})
    axs = sns.lineplot(x=y_true.index, y=y_true.values, color=colors[0], label='True values')
    axs = sns.lineplot(x=y_true.index, y=y_pred, color=colors[1], label='Predictions', ax=axs)    
    plt.tight_layout()
    plt.show()

## Baseline

In [ ]:
dr = DummyRegressor(strategy='mean')
dr.fit(X_train, y_train)

y_pred = dr.predict(X_test)
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}')
print(f'MAE:  {mean_absolute_error(y_test, y_pred):.2f}')
print(f'R2:   {r2_score(y_test, y_pred):.2f}')

line_plot(y_train, y_pred)

## ElasticNet

In [ ]:
scores = cross_val_score(ElasticNet(), X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)
mean = rmse_scores.mean()
std  = rmse_scores.std()
print(f'Mean RMSE: {mean:.2f}')
print(f'Std  RMSE: {std:.2f}')

## SVR

In [ ]:
scores = cross_val_score(SVR(), X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)
mean = rmse_scores.mean()
std  = rmse_scores.std()
print(f'Mean RMSE: {mean:.2f}')
print(f'Std  RMSE: {std:.2f}')

## ExtraTreesRegressor

In [ ]:
scores = cross_val_score(ExtraTreesRegressor(), X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)
mean = rmse_scores.mean()
std  = rmse_scores.std()
print(f'Mean RMSE: {mean:.2f}')
print(f'Std  RMSE: {std:.2f}')

## XGBRegressor

In [ ]:
scores = cross_val_score(XGBRegressor(), X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)
mean = rmse_scores.mean()
std  = rmse_scores.std()
print(f'Mean RMSE: {mean:.2f}')
print(f'Std RMSE : {std:.2f}')

## Hyperparameter tuning

The best models are:
1. SGBRegressor
2. ExtraTreesRegressor

In [ ]:
params = {'n_estimators': np.arange(100, 1000, 10),
          'criterion': ['squared_error', 'friedman_mse', 'poisson', 'absolute_error'],
          'max_depth': np.arange(10, 40, 10),
          'min_samples_split': np.arange(2, 10, 2),
          'min_samples_leaf': np.arange(1, 8, 2),
          'min_weight_fraction_leaf': np.arange(0, 0.6, 0.1),
          'max_features': ['sqrt', 'log2'],
          'max_leaf_nodes': np.arange(10, 60, 10),
          'min_impurity_decrease': np.arange(0, 1, 0.2),
          'bootstrap': [True],
          'oob_score': [False, True],
          'ccp_alpha': np.arange(0, 1, 0.2),
          'max_samples': np.arange(0, 1, 0.1),}

rs = RandomizedSearchCV(ExtraTreesRegressor(), params, scoring='neg_mean_squared_error', n_iter=10, cv=5, verbose=False)
rs.fit(X_train, y_train)

et_best = clone(rs.best_estimator_)
et_best

In [ ]:
params = {'n_estimators': np.arange(100, 600, 100),
          'learning_rate': np.arange(0.01, 0.9, 0.1),
          'gamma': np.arange(0.01, 0.9, 0.1),
          'booster': ['gbtree', 'dart'],
          'max_depth': np.arange(6, 20, 2),
          'max_leaves': np.arange(6, 40, 2),
          'min_child_weight': np.arange(0, 20, 2),}

rs = RandomizedSearchCV(XGBRegressor(), params, scoring='neg_mean_squared_error', n_iter=10, cv=5, verbose=True)
rs.fit(X_train, y_train)

xgb_best = clone(rs.best_estimator_)
xgb_best